In [ ]:
import os
import cv2
import numpy as np
import glob
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
glaucoma_images = [cv2.imread(file, cv2.IMREAD_COLOR) for file in glob.glob("/kaggle/input/drishti/Drishti/Training/Images/GLAUCOMA/*.png")]

normal_images = [cv2.imread(file, cv2.IMREAD_COLOR) for file in glob.glob("/kaggle/input/drishti/Drishti/Training/Images/NORMAL/*.png")]

In [ ]:
print(len(glaucoma_images))
print(len(normal_images))

In [ ]:
new_width = 500
new_height = 500

# Create separate lists for interpolated glaucoma and normal images
interpolated_glaucoma_images = []
interpolated_normal_images = []

# Function to perform bilinear interpolation on an image
def bilinear_interpolation(image, width, height):
    return cv2.resize(image, (width, height), interpolation=cv2.INTER_LINEAR)

for image in glaucoma_images:
    interpolated_image = bilinear_interpolation(image, new_width, new_height)

    interpolated_glaucoma_images.append(interpolated_image)

for image in normal_images:
    # Perform bilinear interpolation
    interpolated_image = bilinear_interpolation(image, new_width, new_height)

    interpolated_normal_images.append(interpolated_image)

print("Bilinear interpolation completed.")

In [ ]:
def histogram_equalization_color(image):
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    l_channel, a_channel, b_channel = cv2.split(lab_image)

    l_channel_equalized = cv2.equalizeHist(l_channel)

    equalized_lab_image = cv2.merge((l_channel_equalized, a_channel, b_channel))

    equalized_bgr_image = cv2.cvtColor(equalized_lab_image, cv2.COLOR_LAB2BGR)

    equalized_grayscale_image = cv2.cvtColor(equalized_bgr_image, cv2.COLOR_BGR2GRAY)

    return equalized_grayscale_image

equalized_interpolated_glaucoma_images = [histogram_equalization_color(image) for image in interpolated_glaucoma_images]

equalized_interpolated_normal_images = [histogram_equalization_color(image) for image in interpolated_normal_images]

print("Histogram Equalization and Conversion to Grayscale completed.")

In [ ]:
import random

num_images_to_display = 5

# Randomly select 'num_images_to_display' images from each list
selected_glaucoma_images = random.sample(equalized_interpolated_glaucoma_images, num_images_to_display)
selected_normal_images = random.sample(equalized_interpolated_normal_images, num_images_to_display)

fig, axes = plt.subplots(2, num_images_to_display, figsize=(15, 5))

for i in range(num_images_to_display):
    axes[0, i].imshow(cv2.cvtColor(selected_glaucoma_images[i], cv2.COLOR_BGR2RGB))
    axes[0, i].set_title("Glaucoma")
    axes[0, i].axis('off')

for i in range(num_images_to_display):
    axes[1, i].imshow(cv2.cvtColor(selected_normal_images[i], cv2.COLOR_BGR2RGB))
    axes[1, i].set_title("Normal")
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

**Augmentation**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
desired_height = 500
desired_width = 500

def augment_images(image_list, augments_per_image, max_augmented_images):
    augmented_images = []
    
    for img in image_list:
        img = cv2.resize(img, (desired_width, desired_height))
        
        img = img.reshape((1, desired_height, desired_width, 1))
        aug_iter = 0
        for aug_img in datagen.flow(img, batch_size=1):
            # Reshape the augmented image back to rank-3
            augmented_images.append(aug_img[0].reshape((desired_height, desired_width)))
            aug_iter += 1
            if aug_iter >= augments_per_image or len(augmented_images) >= max_augmented_images:
                break
                
    return augmented_images

# Initialize a list to store the augmented images
augmented_images = []
max_augmented_images = 40  # Set the maximum number of augmented images

for image in equalized_interpolated_normal_images:
    # Calculate the number of augmentations needed
    remaining_augments = max_augmented_images - len(augmented_images)
    augmentations_needed = min(2, remaining_augments)
    
    augmented = augment_images([image], augmentations_needed, remaining_augments)
    augmented_images.extend(augmented)

    if len(augmented_images) >= max_augmented_images:
        break

print("Image augmentation complete.")

In [ ]:
# X_train = np.array(X_train)
# X_test = np.array(X_test)

# # Convert labels to NumPy arrays (ensure they are integer labels)
# y_train = np.array(y_train)
# y_test = np.array(y_test)

In [ ]:
# print(len(X_test))
# print(len(X_train))
# print(len(y_test))
# print(len(y_train))

In [ ]:
from sklearn.model_selection import train_test_split

# Create a list with labels (1 for glaucomous, 0 for n ormal)
labels = [1] * len(glaucoma_images) + [0] * len(normal_images) + [0] * len(augmented_images)

all_images = equalized_interpolated_glaucoma_images + equalized_interpolated_normal_images + augmented_images

# Split the data into training and testing sets (e.g., 80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(all_images, labels, test_size=0.2, random_state=42, stratify=labels)

X_train = np.array(X_train)
X_test = np.array(X_test)

# Convert labels to NumPy arrays (ensure they are integer labels)
y_train = np.array(y_train)
y_test = np.array(y_test)
print(X_train)



Unet Model

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

def build_unet(input_shape):
    inputs = Input(shape=input_shape)  # Specify the input shape with batch size dimension
    
    # Encoder
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    # Bottom layer
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    
    # Decoder
    up1 = concatenate([UpSampling2D(size=(2, 2))(conv2), conv1], axis=-1)
    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
    
    # Global average pooling layer
    global_pooling = GlobalAveragePooling2D()(conv3)
    
    # Dense layers for classification
    dense1 = Dense(64, activation='relu')(global_pooling)
    outputs = Dense(1, activation='sigmoid')(dense1)
    
    # Create model
    model = Model(inputs=inputs, outputs=outputs)
    return model


# Modify input shape
input_shape = X_train.shape[1:] + (1,)   # Shape of one input image

# Build the model
model = build_unet(input_shape)  # Assuming the shape is (500, 500)
#input_shape_with_batch = (None,) + input_shape  # Adding batch size dimension

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()


Traning the model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


# Fit the model with early stopping callback
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Access training history
print(history.history)


Evaluating the model

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

In [ ]:
from sklearn.metrics import roc_curve, auc

train_predictions = model.predict(X_train)  
test_predictions = model.predict(X_test)   

# Calculate the ROC curve and AUC for the training data
fpr_train, tpr_train, _ = roc_curve(y_train, train_predictions)
roc_auc_train = auc(fpr_train, tpr_train)

# Calculate the ROC curve and AUC for the testing data
fpr_test, tpr_test, _ = roc_curve(y_test, test_predictions)
roc_auc_test = auc(fpr_test, tpr_test)

# Plot the ROC curves
plt.figure(figsize=(8, 6))
plt.plot(fpr_train, tpr_train, color='darkorange', lw=2, label=f'Training ROC curve (area = {roc_auc_train:.2f})')
plt.plot(fpr_test, tpr_test, color='cornflowerblue', lw=2, label=f'Testing ROC curve (area = {roc_auc_test:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Drishti-GS')
plt.legend(loc='lower right')
plt.show()

In [ ]:
y_pred_binary = (test_predictions > 0.5).astype(int)  # Convert probabilities to binary predictions

Confusion Matrix

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred_binary)

# True Positive, True Negative, False Positive, False Negative
tp = conf_matrix[1, 1]
tn = conf_matrix[0, 0]
fp = conf_matrix[0, 1]
fn = conf_matrix[1, 0]

In [ ]:
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
precision = tp / (tp + fp)
recall = sensitivity

# F1 Score
f1_score = 2 * (precision * recall) / (precision + recall)

# Print the metrics
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")

Classification Report

In [ ]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred_binary))